In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  


In [2]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Using GPUs:", gpus)
    except RuntimeError as e:
        print("Error initializing GPUs:", e)
else:
    print("No GPUs available.")


Using GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [5]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import numpy as np
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split

In [6]:
def build_colorization_model(input_shape=(256, 256, 1)):
    inputs = Input(shape=input_shape)
    x = Lambda(lambda x: x * 2 - 1)(inputs)
    
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    block1 = x
    x = MaxPooling2D()(x)
    
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    block2 = x
    x = MaxPooling2D()(x)
    
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    block3 = x
    x = MaxPooling2D()(x)
    
    x = Conv2D(512, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = UpSampling2D()(x)
    x = Concatenate()([x, block3])
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = UpSampling2D()(x)
    x = Concatenate()([x, block2])
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    x = UpSampling2D()(x)
    x = Concatenate()([x, block1])
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    
    outputs = Conv2D(2, 1, padding='same', activation='sigmoid')(x)
    outputs = Lambda(lambda x: x * 2 - 1)(outputs)
    
    return Model(inputs=inputs, outputs=outputs)

In [7]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at {image_path}")
    img = cv2.resize(img, (256, 256))
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l_channel = lab_img[:, :, 0] / 255.0
    return np.expand_dims(l_channel, axis=[0, -1])

def postprocess_lab_to_rgb(L, ab_channels):
    L = L[0, :, :, 0] * 255.0
    a = (ab_channels[0, :, :, 0] + 1) * 127.5
    b = (ab_channels[0, :, :, 1] + 1) * 127.5
    lab_image = np.stack([L, a, b], axis=-1).astype(np.float32)
    rgb_image = cv2.cvtColor(lab_image.astype(np.float32), cv2.COLOR_LAB2RGB)
    rgb_image = cv2.convertScaleAbs(rgb_image, alpha=1.2, beta=5)
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV).astype(np.float32)
    hsv_image[:, :, 1] = hsv_image[:, :, 1] * 1.4
    hsv_image = np.clip(hsv_image, 0, 255).astype(np.uint8)
    rgb_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    return rgb_image

def custom_loss():
    def loss(y_true, y_pred):
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        tv_loss = tf.reduce_mean(tf.image.total_variation(y_pred))
        return mse + 0.001 * tv_loss
    return loss

def load_and_preprocess_data(data_dir):
    image_files = [f for f in os.listdir(data_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    X = []
    Y = []
    
    for img_file in image_files:
        img_path = os.path.join(data_dir, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, (256, 256))
            lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
            
            l_channel = lab_img[:, :, 0]
            ab_channels = lab_img[:, :, 1:]
            
            X.append(l_channel.reshape(256, 256, 1))
            Y.append(ab_channels)
    
    return np.array(X) / 255.0, (np.array(Y) - 128) / 127.0

def prepare_dataset(data_dir):
    X, Y = load_and_preprocess_data(data_dir)
    
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=0.7, random_state=42)
    X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42)
    
    return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test)


In [10]:
def train_model(data_dir, output_dir, batch_size=32, epochs=20):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    (X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = prepare_dataset(data_dir)
    
    model = build_colorization_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 loss=custom_loss(),
                 metrics=['mae'])
    
    callbacks = [
        ModelCheckpoint(os.path.join(output_dir, 'best_model.keras'), 
                       save_best_only=True, 
                       monitor='val_loss'),
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
    ]
    
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    
    history = model.fit(
        datagen.flow(X_train, Y_train, batch_size=batch_size),
        validation_data=(X_val, Y_val),
        epochs=epochs,
        callbacks=callbacks
    )
    
    model.save(os.path.join(output_dir, 'final_model.keras'))
    test_loss = model.evaluate(X_test, Y_test)
    print(f'Test Loss: {test_loss[0]:.4f}')
    print(f'Test MAE: {test_loss[1]:.4f}')
    
    return model, history

In [14]:
if __name__ == "__main__":
    DATA_DIR = "/kaggle/input/old-images"
    OUTPUT_DIR = "/kaggle/working/"
    
    model, history = train_model(DATA_DIR, OUTPUT_DIR)
    

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1730555628.956573     126 service.cc:145] XLA service 0x7f2c1040c6f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730555628.956644     126 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730555628.956648     126 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2024-11-02 13:53:59.195770: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng12{k11=2} for conv (f32[32,128,128,128]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,128,128]{3,2,1,0}, f32[128,128,3,3]

24/24 ━━━━━━━━━━━━━━━━━━━━ 211s 4s/step - loss: 18.8726 - mae: 0.8023 - val_loss: 3.3210 - val_mae: 0.9757 - learning_rate: 0.0010
Epoch 2/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 3.1433 - mae: 0.8348 - val_loss: 4.4805 - val_mae: 0.9316 - learning_rate: 0.0010
Epoch 3/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - loss: 2.3926 - mae: 0.7574 - val_loss: 3.2632 - val_mae: 0.8050 - learning_rate: 0.0010
Epoch 4/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 1.9298 - mae: 0.6381 - val_loss: 1.0431 - val_mae: 0.5454 - learning_rate: 0.0010
Epoch 5/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 1.1672 - mae: 0.4319 - val_loss: 1.0513 - val_mae: 0.4359 - learning_rate: 0.0010
Epoch 6/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 0.7743 - mae: 0.2823 - val_loss: 0.6600 - val_mae: 0.3126 - learning_rate: 0.0010
Epoch 7/20
24/24 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - loss: 0.6252 - mae: 0.2007 - val_loss: 1.5874 - val_mae: 0.2970 - learning_rate: 0.0010
Epoch 8/20
24/24 ━━━━━━━━━━━━

In [35]:
import tensorflow as tf
import numpy as np
from PIL import Image
import cv2
import os
from tensorflow.keras.models import load_model

@tf.keras.utils.register_keras_serializable(name='CustomColorLoss')
class CustomColorLoss(tf.keras.losses.Loss):
    def __init__(self, name='custom_color_loss', **kwargs):
        super().__init__(name=name, **kwargs)
        
    def call(self, y_true, y_pred):
        mse = tf.reduce_mean(tf.square(y_true - y_pred))
        # Add total variation for smoothness
        tv_loss = tf.reduce_mean(tf.image.total_variation(y_pred))
        return mse + 0.001 * tv_loss

def preprocess_image(image_path):
    img = cv2.imread(image_path)
    print(f"Original image shape: {img.shape}")
    
    img = cv2.resize(img, (256, 256))
    print(f"Resized image shape: {img.shape}")
    
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    l_channel = lab_img[:, :, 0].astype(np.float32) / 255.0
    
    print(f"L channel stats - min: {l_channel.min():.3f}, max: {l_channel.max():.3f}")
    return np.expand_dims(l_channel, axis=[0, -1])

def postprocess_lab_to_rgb(L, ab_channels):
    print(f"Input L range: [{L.min():.3f}, {L.max():.3f}]")
    print(f"Input ab range: [{ab_channels.min():.3f}, {ab_channels.max():.3f}]")
    
    L = L[0, :, :, 0] * 100.0
    
    a = ab_channels[0, :, :, 0] * 128.0
    b = ab_channels[0, :, :, 1] * 128.0
    
    lab_image = np.stack([L, a, b], axis=-1).astype(np.float32)
    
    lab_image[:, :, 0] = np.clip(lab_image[:, :, 0], 0, 100)
    lab_image[:, :, 1:] = np.clip(lab_image[:, :, 1:], -128, 127)
    
    print("LAB ranges after scaling:")
    print(f"L: [{lab_image[:,:,0].min():.1f}, {lab_image[:,:,0].max():.1f}]")
    print(f"a: [{lab_image[:,:,1].min():.1f}, {lab_image[:,:,1].max():.1f}]")
    print(f"b: [{lab_image[:,:,2].min():.1f}, {lab_image[:,:,2].max():.1f}]")
    
    rgb_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2RGB)
    
    rgb_image = np.clip(rgb_image * 255.0, 0, 255).astype(np.uint8)
    
    rgb_image = cv2.convertScaleAbs(rgb_image, alpha=1.1, beta=5)
    
    hsv_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV).astype(np.float32)
    hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] * 1.2, 0, 255)
    hsv_image = hsv_image.astype(np.uint8)
    rgb_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    
    return rgb_image

test_image_path = "/kaggle/input/old-images/1054.jpg"
output_image_path = "/kaggle/working/colored_1054.jpg"
model_path = os.path.join(OUTPUT_DIR, 'best_model.keras')

print("Loading model...")
try:
    model = load_model(model_path, 
                      custom_objects={'CustomColorLoss': CustomColorLoss,
                                    'custom_color_loss': CustomColorLoss()})
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {str(e)}")
    raise

print("\nProcessing image...")
img_array = preprocess_image(test_image_path)
print(f"Input array shape: {img_array.shape}")

print("\nGenerating color predictions...")
ab_channels = model.predict(img_array)
print(f"Predictions shape: {ab_channels.shape}")
print(f"Predictions range: [{ab_channels.min():.3f}, {ab_channels.max():.3f}]")

print("\nConverting to RGB...")
rgb_image = postprocess_lab_to_rgb(img_array, ab_channels)
print(f"Final RGB shape: {rgb_image.shape}")
print(f"Final RGB range: [{rgb_image.min()}, {rgb_image.max()}]")

Image.fromarray(rgb_image).save(output_image_path)
print("\nImage saved successfully")

Loading model...
Model loaded successfully

Processing image...
Original image shape: (373, 600, 3)
Resized image shape: (256, 256, 3)
L channel stats - min: 0.004, max: 1.000
Input array shape: (1, 256, 256, 1)

Generating color predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step
Predictions shape: (1, 256, 256, 2)
Predictions range: [0.051, 0.179]

Converting to RGB...
Input L range: [0.004, 1.000]
Input ab range: [0.051, 0.179]
LAB ranges after scaling:
L: [0.4, 100.0]
a: [21.8, 22.9]
b: [6.6, 8.0]
Final RGB shape: (256, 256, 3)
Final RGB range: [0, 255]

Image saved successfully
